от сюда взял
https://towardsdatascience.com/implementing-word2vec-in-pytorch-skip-gram-model-e6bae040d2fb

перспективная ссылка
https://notebook.community/Diyago/Machine-Learning-scripts/DEEP%20LEARNING/Pytorch%20from%20scratch/word2vec-embeddings/Negative_Sampling

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F

In [38]:
# corpus = [
#     'he is a king',
#     'she is a queen',
#     'he is a man',
#     'she is a woman',
#     'warsaw is poland capital',
#     'berlin is germany capital',
#     'paris is france capital',   
# ]

In [39]:


# def tokenize_corpus(corpus):
#     tokens = [x.split() for x in corpus]
#     return tokens

# tokenized_corpus = tokenize_corpus(corpus)

In [40]:

text = '''As the eight strange beings applauded, one of them even cupping a hand over her lipsticked mouth to cheer, Joel tried to grasp what was happening. The nine of them sat in a fire rimmed cavern around a conference table shaped from warm volcanic rock. A chandelier of human bones dangled from the cavern’s ceiling, and it rattled around at random like wind chimes. A massive goat-man with reddish-black skin and wicked horns on his head towered above the seven others, who flanked him to either side. They looked like pure stereotype. A fat slob with sixteen chins, a used car saleman looking guy with gold and silver jewelry all over him, a sultry dominatrix in skin tight leather. On the other side a disheveled looking college drop out, a pretty boy staring in a mirror, a bald, muscular dude who looked like someone’s pissed off step-dad and a sour faced woman glancing jealously around the room. Just where the hell was he? Joel concentrated on his last memory. He remembered highlighting pages as his private jet, “The Holy Gust,” flew over the sapphire waters of the Bahamas. He had been reviewing his sermon for Sunday – dotting the I’s and crossing the crosses, a little god humor there, praise him – and the pilot’s voice had crackled over the intercom about turbulence. Kimberly, his personal assistant, had taken his plow out of her mouth and put on her seat belt. The plane had shook and then'''.lower()

words = text.split()
vocab = set(words)
tokenized_corpus = vocab
word2idx = {w: idx for (idx, w) in enumerate(vocab)}
idx2word = {idx: w for (idx, w) in enumerate(vocab)}
vocabulary_size = len(vocab)

In [41]:
# vocabulary = []
# for sentence in tokenized_corpus:
#     for token in sentence:
#         if token not in vocabulary:
#             vocabulary.append(token)

# word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
# idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

# vocabulary_size = len(vocabulary)

In [52]:
window_size = 10
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [56]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [57]:
embedding_dims = 300
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 10
learning_rate = 0.01
kek = []
for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.item()
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
        kek.append(log_softmax.detach().numpy())
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')
kek = z2.detach().numpy()
print(kek[1])

word2idx_l = dict(zip(vocab, kek))
print(word2idx_l)
# word2idx_l = {w: kek for ([kek[i] for i in range(len(kek))], w) in enumerate(vocabulary)}
# # word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}

TypeError: new(): invalid data type 'str'

зашло с 14-ой попытки

window = 10
lr = 0.01
negative sampling
epoch = 5
dims = 300